In [1]:
liked_books = ["22822858","6327123","29430012","160296","19181926"]

In [3]:
!head book_id_map.csv
#json with book metadata and csv file with interaction and ratings for the books have different IDs
#this map will map between one ID and the other

book_id_csv,book_id
0,34684622
1,34536488
2,34017076
3,71730
4,30422361
5,33503613
6,33517540
7,34467031
8,6383669


In [12]:
csv_book_mapping = {}
with open("book_id_map.csv", "r") as f:
    while True:
        line = f.readline()
        if not line:
            break
        csv_id, book_id = line.strip().split(",")
        csv_book_mapping[csv_id] = book_id

In [5]:
len(csv_book_mapping)

2360651

In [6]:
!wc -l goodreads_interactions.csv

 228648343 goodreads_interactions.csv


# Find users who like the same books as us

In [7]:
!head goodreads_interactions.csv

user_id,book_id,is_read,rating,is_reviewed
0,948,1,5,0
0,947,1,5,1
0,946,1,5,0
0,945,1,5,0
0,944,1,5,0
0,943,1,5,0
0,942,1,5,0
0,941,1,5,0
0,940,1,5,0


In [13]:
# this set will contain users that have read and liked books similar to us
overlap_users = set()

with open("goodreads_interactions.csv",'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        user_id, csv_id, _, rating, _ = line.split(",")
        
        if user_id in overlap_users:
            continue
            
        try:
            rating = int(rating)
        except ValueError:
            continue
            
        book_id = csv_book_mapping[csv_id]
        
        if book_id in liked_books and rating >= 4:
            overlap_users.add(user_id)

# Finding out what books those users liked

In [14]:
rec_lines = []

with open("goodreads_interactions.csv",'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        user_id, csv_id, _, rating, _ = line.split(",")
        
        if user_id in overlap_users:
            book_id = csv_book_mapping[csv_id]
            rec_lines.append([user_id, book_id, rating])

In [15]:
len(overlap_users)

10021

In [16]:
len(rec_lines)

6164348

In [17]:
#rank the recommendations

import pandas as pd

recs = pd.DataFrame(rec_lines, columns = ["user_id", "book_id", "rating"])
recs["book_id"] = recs["book_id"].astype(str)

In [23]:
top_recs = recs["book_id"].value_counts().head(10)
top_recs = top_recs.index.values

In [20]:
books_titles = pd.read_json("books_titles.json")
books_titles["book_id"] = books_titles["book_id"].astype(str)

In [24]:
books_titles.head()

,book_id,title,ratings,url,cover_image,mod_title
0,7327624,"The Unschooled Wizard (Sun Wolf and Starhawk, ...",140,https://www.goodreads.com/book/show/7327624-th...,https://images.gr-assets.com/books/1304100136m...,the unschooled wizard sun wolf and starhawk 12
1,6066819,Best Friends Forever,51184,https://www.goodreads.com/book/show/6066819-be...,https://s.gr-assets.com/assets/nophoto/book/11...,best friends forever
2,287141,The Aeneid for Boys and Girls,46,https://www.goodreads.com/book/show/287141.The...,https://s.gr-assets.com/assets/nophoto/book/11...,the aeneid for boys and girls
3,6066812,All's Fairy in Love and War (Avalon: Web of Ma...,98,https://www.goodreads.com/book/show/6066812-al...,https://images.gr-assets.com/books/1316637798m...,alls fairy in love and war avalon web of magic 8
4,287149,The Devil's Notebook,986,https://www.goodreads.com/book/show/287149.The...,https://images.gr-assets.com/books/1328768789m...,the devils notebook


# Creating Initial Recs

In [26]:
books_titles[books_titles["book_id"].isin(top_recs)]

,book_id,title,ratings,url,cover_image,mod_title
53027,77203,The Kite Runner,1848782,https://www.goodreads.com/book/show/77203.The_...,https://images.gr-assets.com/books/1484565687m...,the kite runner
284473,2767052,"The Hunger Games (The Hunger Games, #1)",4899965,https://www.goodreads.com/book/show/2767052-th...,https://images.gr-assets.com/books/1447303603m...,the hunger games the hunger games 1
373254,19063,The Book Thief,1193697,https://www.goodreads.com/book/show/19063.The_...,https://images.gr-assets.com/books/1390053681m...,the book thief
463463,4671,The Great Gatsby,2758812,https://www.goodreads.com/book/show/4671.The_G...,https://images.gr-assets.com/books/1490528560m...,the great gatsby
790927,2657,To Kill a Mockingbird,3255518,https://www.goodreads.com/book/show/2657.To_Ki...,https://images.gr-assets.com/books/1361975680m...,to kill a mockingbird
809846,17333223,The Goldfinch,410269,https://www.goodreads.com/book/show/17333223-t...,https://images.gr-assets.com/books/1451554970m...,the goldfinch
878151,18143977,All the Light We Cannot See,498685,https://www.goodreads.com/book/show/18143977-a...,https://images.gr-assets.com/books/1451445646m...,all the light we cannot see
878545,3,Harry Potter and the Sorcerer's Stone (Harry P...,4765497,https://www.goodreads.com/book/show/3.Harry_Po...,https://images.gr-assets.com/books/1474154022m...,harry potter and the sorcerers stone harry pot...
993069,22822858,A Little Life,72278,https://www.goodreads.com/book/show/22822858-a...,https://images.gr-assets.com/books/1446469353m...,a little life
1091642,22557272,The Girl on the Train,1076144,https://www.goodreads.com/book/show/22557272-t...,https://images.gr-assets.com/books/1490903702m...,the girl on the train


the books above are books that people who have rated similarly your fav books also rated highly. however as you can see they are all highly popular books. we want books that are less popular.

In [27]:
#gives how many times each book appeared in our set
all_recs = recs["book_id"].value_counts()

In [28]:
all_recs

22822858    7294
18143977    6213
2657        6031
22557272    5966
4671        5623
            ... 
15833498       1
15833495       1
15833493       1
18622967       1
13815558       1
Name: book_id, Length: 632847, dtype: int64

In [29]:
# convert to data frame
all_recs = all_recs.to_frame().reset_index()

In [30]:
all_recs

,index,book_id
0,22822858,7294
1,18143977,6213
2,2657,6031
3,22557272,5966
4,4671,5623
...,...,...
632842,15833498,1
632843,15833495,1
632844,15833493,1
632845,18622967,1


In [31]:
all_recs.columns = ["book_id","book_count"]

In [32]:
all_recs

,book_id,book_count
0,22822858,7294
1,18143977,6213
2,2657,6031
3,22557272,5966
4,4671,5623
...,...,...
632842,15833498,1
632843,15833495,1
632844,15833493,1
632845,18622967,1


In [33]:
all_recs = all_recs.merge(books_titles, how = "inner", on="book_id")

In [34]:
all_recs

,book_id,book_count,title,ratings,url,cover_image,mod_title
0,22822858,7294,A Little Life,72278,https://www.goodreads.com/book/show/22822858-a...,https://images.gr-assets.com/books/1446469353m...,a little life
1,18143977,6213,All the Light We Cannot See,498685,https://www.goodreads.com/book/show/18143977-a...,https://images.gr-assets.com/books/1451445646m...,all the light we cannot see
2,2657,6031,To Kill a Mockingbird,3255518,https://www.goodreads.com/book/show/2657.To_Ki...,https://images.gr-assets.com/books/1361975680m...,to kill a mockingbird
3,22557272,5966,The Girl on the Train,1076144,https://www.goodreads.com/book/show/22557272-t...,https://images.gr-assets.com/books/1490903702m...,the girl on the train
4,4671,5623,The Great Gatsby,2758812,https://www.goodreads.com/book/show/4671.The_G...,https://images.gr-assets.com/books/1490528560m...,the great gatsby
...,...,...,...,...,...,...,...
547559,15833498,1,"Soul Eater, Vol. 22 (Soul Eater, #22)",881,https://www.goodreads.com/book/show/15833498-s...,https://images.gr-assets.com/books/1345230011m...,soul eater vol 22 soul eater 22
547560,15833495,1,"Soul Eater, Vol. 21 (Soul Eater, #21)",806,https://www.goodreads.com/book/show/15833495-s...,https://images.gr-assets.com/books/1345229933m...,soul eater vol 21 soul eater 21
547561,15833493,1,"Soul Eater, Vol. 20 (Soul Eater, #20)",823,https://www.goodreads.com/book/show/15833493-s...,https://images.gr-assets.com/books/1345229889m...,soul eater vol 20 soul eater 20
547562,18622967,1,Ice and A Slice,23,https://www.goodreads.com/book/show/18622967-i...,https://s.gr-assets.com/assets/nophoto/book/11...,ice and a slice


In [35]:
#we're looking for books that are popular amongst users like us who like similar books
# and penalising those that are also popular more generally
all_recs["score"] = all_recs["book_count"] * (all_recs)["book_count"]/all_recs["ratings"]

In [36]:
all_recs.sort_values("score", ascending=False).head(10)

,book_id,book_count,title,ratings,url,cover_image,mod_title,score
0,22822858,7294,A Little Life,72278,https://www.goodreads.com/book/show/22822858-a...,https://images.gr-assets.com/books/1446469353m...,a little life,736.080633
3907,28170940,214,"Lethal White (Cormoran Strike, #4)",106,https://www.goodreads.com/book/show/28170940-l...,https://s.gr-assets.com/assets/nophoto/book/11...,lethal white cormoran strike 4,432.037736
17,29430012,4235,A Gentleman in Moscow,44419,https://www.goodreads.com/book/show/29430012-a...,https://images.gr-assets.com/books/1459524472m...,a gentleman in moscow,403.773723
6840,9624993,126,"The Mirror and the Light (Thomas Cromwell, #3)",40,https://www.goodreads.com/book/show/9624993-th...,https://s.gr-assets.com/assets/nophoto/book/11...,the mirror and the light thomas cromwell 3,396.900000
365,16126596,1374,The People in the Trees,7644,https://www.goodreads.com/book/show/16126596-t...,https://images.gr-assets.com/books/1356108388m...,the people in the trees,246.974882
571,32920226,1045,"Sing, Unburied, Sing",4592,https://www.goodreads.com/book/show/32920226-s...,https://images.gr-assets.com/books/1499340866m...,sing unburied sing,237.810322
7191,34927828,120,The Great Alone,70,https://www.goodreads.com/book/show/34927828-t...,https://images.gr-assets.com/books/1501852384m...,the great alone,205.714286
563,25810500,1057,What is Not Yours is Not Yours,5470,https://www.goodreads.com/book/show/25810500-w...,https://images.gr-assets.com/books/1447463920m...,what is not yours is not yours,204.250274
539,17912498,1087,The Queen of the Night,6115,https://www.goodreads.com/book/show/17912498-t...,https://images.gr-assets.com/books/1460425080m...,the queen of the night,193.224693
71,27071490,3067,Homegoing,49315,https://www.goodreads.com/book/show/27071490-h...,https://images.gr-assets.com/books/1448108591m...,homegoing,190.742959


In [37]:
popular_recs = all_recs[all_recs["book_count"] > 75].sort_values("score", ascending=False)

In [38]:
def make_clickable(val):
    return '<a target = "_blank" href="{}">Goodreads</a>'.format(val)

def show_image(val):
    return '<img src="{}" width=50></img>'.format(val)

popular_recs[~popular_recs["book_id"].isin(liked_books)].head(10).style.format({'url':make_clickable, 'cover_image':show_image})

,book_id,book_count,title,ratings,url,cover_image,mod_title,score
3907,28170940,214,"Lethal White (Cormoran Strike, #4)",106,Goodreads,,lethal white cormoran strike 4,432.037736
6840,9624993,126,"The Mirror and the Light (Thomas Cromwell, #3)",40,Goodreads,,the mirror and the light thomas cromwell 3,396.900000
365,16126596,1374,The People in the Trees,7644,Goodreads,,the people in the trees,246.974882
571,32920226,1045,"Sing, Unburied, Sing",4592,Goodreads,,sing unburied sing,237.810322
7191,34927828,120,The Great Alone,70,Goodreads,,the great alone,205.714286
563,25810500,1057,What is Not Yours is Not Yours,5470,Goodreads,,what is not yours is not yours,204.250274
539,17912498,1087,The Queen of the Night,6115,Goodreads,,the queen of the night,193.224693
71,27071490,3067,Homegoing,49315,Goodreads,,homegoing,190.742959
10646,26856502,81,"Vengeful (Villains, #2)",35,Goodreads,,vengeful villains 2,187.457143
437,29983711,1233,Pachinko,8161,Goodreads,,pachinko,186.287097
